In [1]:
import numpy as np
import openseespy.opensees as ops
import opsvis as opsv
import matplotlib.pyplot as plt
import os

In [2]:
ops.wipe()
ops.model('basic', '-ndm', 3, '-ndf', 6)

In [3]:
nodearray=np.loadtxt('C:\\Users\\Ashen\\Desktop\\Topology\\Demo\\Node.txt',delimiter=',')
elementarray=np.loadtxt('C:\\Users\\Ashen\\Desktop\\Topology\\Demo\\Element.txt',delimiter=',')
thickness=10

for index in range(nodearray.shape[0]):
    nodeTag=int(nodearray[index][0])
    nodeX=float(nodearray[index][1])
    nodeY=float(nodearray[index][2])
    nodeZ=float(nodearray[index][3])
    ops.node(nodeTag,nodeX,nodeY,nodeZ)

Top=nodearray[np.where(nodearray[:,2]==150)]
Bottom=nodearray[np.where(nodearray[:,2]==-150)]
Other=nodearray[np.where((-150<nodearray[:,2]) & (nodearray[:,2]<150))]

for index in range(Top.shape[0]):
    nodeTag=int(Top[index][0])
    print(nodeTag)
    ops.fix(nodeTag,0,0,1,1,1,0)
    # ops.fix(nodeTag,0,1,1,1,1,1)

for index in range(Other.shape[0]):
    nodeTag=int(Other[index][0])
    ops.fix(nodeTag,0,0,1,1,1,0)

for index in range(Bottom.shape[0]):
    nodeTag=int(Bottom[index][0])
    ops.fix(nodeTag,1,1,1,1,1,1)

IDJ2=1
ops.nDMaterial('J2Plasticity', 1, 210000, 210000/(2*(1+0.3)), 235, 235, 0, 0)
ops.nDMaterial('PlateFiber', 2, 1)
ops.section('PlateFiber', IDJ2, 2, thickness)

IDElasticIsotropic=2
ops.nDMaterial('ElasticIsotropic', 3, 100, 0.3)
ops.nDMaterial('PlateFiber', 4, 3)
ops.section('PlateFiber', IDElasticIsotropic, 4, thickness)

for index in range(elementarray.shape[0]):
    elementTag=int(elementarray[index][0])
    element1=int(elementarray[index][1])
    element2=int(elementarray[index][2])
    element3=int(elementarray[index][3])
    element4=int(elementarray[index][4])
    ops.element('ShellMITC4',elementTag,element1,element2,element3,element4,IDJ2)

1
65
129
193
257
321
385
449
513
577
641
705
769
833
897
961
1025
1089
1153
1217
1281
1345
1409
1473
1537
1601
1665
1729
1793
1857
1921
1985
2049


In [4]:
rr=0.1
for iteration in range(11):
    Output_dir = f'C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}'
    if not os.path.exists(Output_dir):
        os.makedirs(Output_dir)

    for index in range(Bottom.shape[0]):
        nodeTag=int(Bottom[index][0])
        ops.recorder('Node','-file',f"C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\Reaction{nodeTag}_{iteration}.txt","-time",'-node',nodeTag,'-dof',1,2,3,'reaction')

    for index in range(Top.shape[0]):
        nodeTag=int(Top[index][0])
        ops.recorder('Node','-file',f"C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\Disp{nodeTag}_{iteration}.txt","-time",'-node',nodeTag,'-dof',1,2,3,'disp')

    ops.recorder('Element','-file',f"C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\Stress_{iteration}.txt","-time",'-eleRange',1,2016,'stresses')
    ops.recorder('Node','-file',f"C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\Displacement_{iteration}.txt","-time",'-nodeRange',1,2112,'-dof',1,2,3,'disp')    

    ops.timeSeries('Linear', 22)
    ops.pattern( "Plain", 200,22)

    for index in range(Top.shape[0]):
        nodeTag=int(Top[index][0])
        ops.load(nodeTag,200,0,0,0,0,0)

    ops.constraints("Plain")
    ops.numberer("RCM")
    ops.system("BandGeneral")
    ops.test('NormDispIncr', 1e-4, 200)
    # ops.algorithm("KrylovNewton")
    ops.algorithm('ExpressNewton',2,1.0,'-currentTangent','-factorOnce')
    ops.integrator("DisplacementControl",961,1,0.1)
    ops.analysis("Static")
    ops.analyze(10)

    ops.remove('recorders')
    ops.reset()
    ops.remove('loadPattern',200)
    ops.reset()
    ops.remove('timeSeries',22)
    ops.reset()
    ops.wipeAnalysis()
    
    stress=np.loadtxt(f'C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\Stress_{iteration}.txt')[:,1:]
    shellstress=[]

    for i in range(2016):
        S11a=stress[:,0+i*32]
        S22a=stress[:,1+i*32]
        S12a=stress[:,2+i*32]

        S11b=stress[:,8+i*32]
        S22b=stress[:,9+i*32]
        S12b=stress[:,10+i*32]

        S11c=stress[:,16+i*32]
        S22c=stress[:,17+i*32]
        S12c=stress[:,18+i*32]

        S11d=stress[:,24+i*32]
        S22d=stress[:,25+i*32]
        S12d=stress[:,26+i*32]

        S11=(S11a+S11b+S11c+S11d)/4
        S22=(S22a+S22b+S22c+S22d)/4
        S12=(S12a+S12b+S12c+S12d)/4
        S=np.sqrt(S11**2-S11*S22+S22**2+3*S12**2)
        shellstress.append(S[-1]/thickness)
    
    stressmax=max(shellstress)

    removeID=np.array([])
    for i in range(2016):
        elementTag=int(elementarray[i][0])
        element1=int(elementarray[i][1])
        element2=int(elementarray[i][2])
        element3=int(elementarray[i][3])
        element4=int(elementarray[i][4])

        if (element1 in Top[:,0]) or (element2 in Top[:,0]) or (element3 in Top[:,0]) or (element4 in Top[:,0]):
            continue
        if shellstress[i]<=rr*stressmax:
            ops.remove('ele',i+1)
            ops.element('ShellMITC4',i+1,element1,element2,element3,element4,IDElasticIsotropic)
            removeID=np.append(removeID,i+1)
    
    np.savetxt(f'C:\\Users\\Ashen\\Desktop\\Topology\\Case1\\{iteration}\\RemoveID.txt',removeID)
    rr+=0.02
    print(f"Loop_{iteration} finished.")

Loop_0 finished.
Loop_1 finished.
Loop_2 finished.
Loop_3 finished.
Loop_4 finished.
Loop_5 finished.
Loop_6 finished.
Loop_7 finished.
Loop_8 finished.
Loop_9 finished.
Loop_10 finished.
